### Installing packages ###

In [1]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import plotly.graph_objects as go
import plotly.subplots as sp
from plotly.subplots import make_subplots

import survivalnet2
from survivalnet2.data.labels import stack_labels, unstack_labels
from survivalnet2.losses import efron, cox
from survivalnet2.metrics.concordance import HarrellsC
from survivalnet2.visualization import km_plot

# Set random seeds for reproducibility
np.random.seed(51)
tf.random.set_seed(51)

### Data preprocessing ###

In [2]:
def binarize_columns(df):
    # binarizr region identifier as 0 or 1
    df[df.columns[0]] = df.iloc[:, 0].apply(lambda x: 1 if x == 'TUMOR' else 0)
    return df

def compute_median_values(data_files):
    # Read in first file to get the columns
    sample = pd.read_csv(data_files[0])
    num_cols = sample.shape[1] - 2  # Exclude the first two columns

    # Initialize array to store values for all files
    values = np.empty((0, num_cols))

    # Iterate over all files and extract values
    for data_file in data_files:
        # Read in data and skip first row (assumed to be header)
        df = pd.read_csv(data_file, skiprows=[0], usecols=range(2, num_cols+2))
        df = binarize_columns(df)
        values = np.concatenate((values, df.values), axis=0)
    
    # Compute median values for each column
    median_values = np.median(values, axis=0)
    median_dict = {sample.columns[i+2]: median_values[i] for i in range(num_cols)}
    return median_dict


def pad_missing_values(df, median_dict):
    # Replace missing values with median value for each column
    for col in df.columns:
        median_value = median_dict[col]
        df[col].fillna(median_value, inplace=True)
    return df


def min_max_normalize_features(df):

    for col in df.columns:
        min_value = df[col].min()
        max_value = df[col].max()
        
        # Check if values are not already in the range [0, 1]
        if (min_value < 0 or max_value > 1):
            df[col] = (df[col] - min_value) / (max_value - min_value)
        
    return df

def create_label_dict(label_dir):
    df = pd.read_csv(label_dir)
    column_names = df.columns  # Get the column names from the first row
    label_dict = {}
    for i in range(1, len(df)):
        name = df.iloc[i, 0]  # Get the sample name from the first column of the current row
        time = df.iloc[i, column_names.get_loc('ClinicalFeats.Survival.BCSS.YearsFromDx')]  # Get the time data from the 'ClinicalFeats.Survival.BCSS.YearsFromDx' column
        event = df.iloc[i, column_names.get_loc('ClinicalFeats.Survival.BCSS')]  # Get the event data from the 'ClinicalFeats.Survival.BCSS' column
        label_dict[name] = (time, event)
    return label_dict

def z_score_normalize_ragged_tensor(ragged_tensor):
    scaler = StandardScaler()

    # Store the lengths of the subtensors before flattening the ragged tensor
    lengths = [len(subtensor) for subtensor in ragged_tensor]

    # Flatten the ragged tensor and convert it to a 2D numpy array
    flat_data = ragged_tensor.to_tensor().numpy().reshape(-1, D)

    # Fit and transform the data using the StandardScaler
    normalized_data = scaler.fit_transform(flat_data)

    # Check if mean is close to 0 and standard deviation is close to 1
    mean = np.mean(normalized_data, axis=0)
    std_dev = np.std(normalized_data, axis=0)

    print("Mean:", mean)
    print("Standard Deviation:", std_dev)
    
    # Restore the original ragged structure
    normalized_data_ragged = []
    start_idx = 0
    for length in lengths:
        normalized_data_ragged.append(normalized_data[start_idx:start_idx+length])
        start_idx += length

    return tf.ragged.constant(normalized_data_ragged, ragged_rank=1, dtype=tf.float32)


### Parameters definition ###

In [4]:
# define dimensionality
D = 49
print(f"Dimensionality of each feature vector is: {D}")

# Define the batch size you want to use
batch_size = 64


data_dir = '/Users/shangke/Desktop/pathology/data/perSlideRegionFeatures/CPSII_40X'
label_dir = '/Users/shangke/Desktop/pathology/data/perSlideRegionFeatures/FusedData_CPSII_40X.csv'
csv_names = os.listdir(data_dir)
null_count = 0
label_dict = create_label_dict(label_dir)
valid_csv_names = []

for name in csv_names:
    if name.rstrip('.csv') in list(label_dict.keys()):
        valid_csv_names.append(name)

    else:
        null_count += 1

data_files = [os.path.join(data_dir, str(csv_name)) for csv_name in valid_csv_names]

print(f"Number of samples with missing label data: {null_count}")


Dimensionality of each feature vector is: 49
Number of samples with missing label data: 54


### Generate dataset ###

In [5]:
def read_data(data_files, label_dict):
    """
    Reads in the data files, binarizes the columns, pads missing values, and normalizes the features.

    Args:
        data_files (list): List of file paths to the data files.
        label_dict (dict): Dictionary mapping file names to (time, event) tuples.

    Returns:
        A tuple containing:
            - rows_tensor (tf.RaggedTensor): A ragged tensor containing the feature vectors for each sample.
            - labels_tensor (tuple): A tuple of two tensors, containing the time and event labels for each sample.
    """
    rows_list = []
    time_list = []
    event_list = []
    empty_count = 0

    # Calculate median values for each feature across all data files
    median_values = compute_median_values(data_files)

    for data_file in data_files:
        # Get the name of the file without the extension
        name = os.path.splitext(os.path.basename(data_file))[0]

        # Skip the file if it is not in the label dictionary
        if name not in label_dict:
            empty_count += 1
            continue

        # Read in the data file
        df = pd.read_csv(data_file)
        

        # Skip the file if it has no rows
        if df.shape[0] < 1:
            empty_count += 1
            print(name)
            continue

        # Binarize the columns and pad missing values
        df = df.iloc[:, 2:]  # Drop the first two columns
        df = binarize_columns(df)
        df = pad_missing_values(df, median_values)

        # Normalize the features
        # df = min_max_normalize_features(df)
    
        # Add the feature vector and labels to the lists
        rows_list.append(df.values)
        time, event = label_dict[name]
        time_list.append(time)
        event_list.append(event)
    
    # Convert the lists to tensors
    rows_tensor = tf.ragged.constant(rows_list, ragged_rank=1, dtype=tf.float32)
    labels_tensor = stack_labels(tf.convert_to_tensor(time_list, dtype=tf.float32),
                                 tf.convert_to_tensor(event_list, dtype=tf.float32))

    print(f"Number of samples: {len(rows_list)}")
    print(f"Number of empty data files: {empty_count}")

    return rows_tensor, labels_tensor

data, labels = read_data(data_files, label_dict)

data = z_score_normalize_ragged_tensor(data)


2023-05-29 11:53:45.316115: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Number of samples: 1654
Number of empty data files: 0
Mean: [-4.4965680e-05  5.5734240e-06 -5.3070180e-06  1.9735680e-05
 -2.5194002e-05 -9.3263006e-06  1.0113984e-05 -4.9780060e-06
 -5.3209010e-06 -2.9716795e-05  2.2781544e-05 -1.0131360e-05
  9.6866052e-06  3.2072548e-05  5.3374402e-05  1.7475038e-05
  1.6078709e-05 -1.9161647e-05  7.4878062e-07  6.7512759e-05
 -1.2746840e-05 -3.0244182e-05  4.2329266e-05 -1.4616140e-05
  3.3741369e-05 -1.1984759e-05  8.5156529e-08 -4.7412617e-05
  4.5339577e-05  4.9911210e-05 -1.4850494e-06  3.0032786e-06
  3.4058627e-05  1.7997475e-06  2.2410493e-05  6.9520884e-06
 -3.3284890e-05  3.9771196e-05  8.6074206e-06 -4.5091103e-05
 -6.9029061e-06  2.0899532e-05 -2.0842950e-05  6.2503772e-05
  5.1923698e-06  1.0593353e-04 -4.8191410e-05 -4.3020114e-05
  2.7419424e-06]
Standard Deviation: [1.0058852  0.9993391  0.9993592  1.0026546  0.9882494  0.99924344
 1.0042104  0.9886866  0.9994165  0.98902804 0.9910845  0.99810594
 1.0031139  1.0016706  1.0060112  0.9

### Remove subjects with persistent NaNs ###

Some NaNs may remain for subjects (10 samples) that have a single region that also contains NaN features (median imputation doesn't work in this case).

In [6]:
# After this thread, the dataset size now is reduced from 1655 to 1654
indices = []
for i, subject in enumerate(data):
    if not np.sum(np.isnan(subject)):
        indices.append(i)
data = tf.gather(data, np.array(indices), axis=0)
labels = tf.gather(labels, np.array(indices), axis=0)

### Data visualization ###

Visualize the distribution of each features and also a distrubution of the number of time & event labels.

In [ ]:
def visualize_data_distribution(data, labels):
    # Flatten the data and create a DataFrame
    flattened_data = data.flat_values.numpy()
    data_df = pd.DataFrame(flattened_data)
    
    # Visualize the distribution of each feature
    n_features = data_df.shape[1]
    fig, axes = plt.subplots(n_features, 1, figsize=(10, 4 * n_features))

    for i in range(n_features):
        sns.kdeplot(data_df.iloc[:, i], ax=axes[i])
        axes[i].set_title(f"Feature {i+1} Distribution")
        axes[i].set_xlabel(f"Feature {i+1}")
        axes[i].set_ylabel("Density")

    plt.tight_layout()
    plt.show()

    # Visualize the distribution of labels
    labels_df = pd.DataFrame(labels.numpy(), columns=["Time", "Event"])
    
    fig, axes = plt.subplots(1, 2, figsize=(10, 4))

    sns.histplot(labels_df["Time"], kde=True, ax=axes[0], bins=50)
    axes[0].set_title("Time Distribution")
    axes[0].set_xlabel("Time")
    axes[0].set_ylabel("Frequency")

    sns.histplot(labels_df["Event"], discrete=True, ax=axes[1], binwidth=1)
    axes[1].set_title("Event Distribution")
    axes[1].set_xlabel("Event")
    axes[1].set_ylabel("Frequency")
    axes[1].set_xticks([0, 1])

    plt.tight_layout()
    plt.show()

visualize_data_distribution(data, labels)

### Attention model architecture ###

In [8]:
def build_model_att(D):
    # Input layer
    inputs = tf.keras.layers.Input(shape=(None, D), ragged=True)

    # Attention weights
    att = tf.keras.layers.Dense(units=1, activation="relu", name="att")(inputs)

    # Normalize weights to sum to 1
    totals = tf.reduce_sum(att, axis=1, name="att_total")
    normalized = tf.math.divide_no_nan(att, tf.expand_dims(totals, axis=1), name="normalized")

    # Use attention weights to calculate weighted sum of regions
    pooled = tf.linalg.matmul(normalized, inputs, transpose_a=True)

    # Remove the ragged dimension and reshape pooled tensor
    pooled = tf.squeeze(pooled.to_tensor(), axis=1)

    # Apply a linear layer to the pooled vector to generate the time and event risk values
    risk = tf.keras.layers.Dense(units=1, activation="linear", name="risk")(pooled)

    # Build the model
    model = tf.keras.models.Model(inputs=inputs, outputs=[risk, normalized])

    print(f"The input shape of model is: {model.input_shape}")
    print(f"The output shape of model is: {model.output_shape}")

    return model

### Averaging model architecture ###

In [7]:
def build_model_avg(D):
    # Input layer
    inputs = tf.keras.layers.Input(shape=(None, D), ragged=True)

    # Average feature vectors along the ragged dimension
    averaged = tf.reduce_mean(inputs, axis=1)

    # Apply a linear layer to the averaged vector to generate the time and event risk values
    risk = tf.keras.layers.Dense(units=1, activation="linear", name="risk")(averaged)

    # Build the model
    model = tf.keras.models.Model(inputs=inputs, outputs=risk)

    print(f"The input shape of model is: {model.input_shape}")
    print(f"The output shape of model is: {model.output_shape}")

    return model

###  Data batching ###

In [9]:
def perform_k_fold_cross_validation(data, labels, batch_size, n_splits, model_name):
    # Initialize KFold
    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)

    for fold, (train_indices, val_indices) in enumerate(kfold.split(data)):
        print(f'Fold {fold + 1}/{n_splits}')
        fold += 1

        if model_name == 'model_att':
            # Create a new model for each fold
            model = build_model_att(D)
        elif model_name == 'model_avg':
            model = build_model_avg(D)
        else:
            raise ValueError("Invalid model name.")  # Add an error to handle unexpected model_name values

        model.compile(
            loss={"risk": cox},
            metrics={"risk": HarrellsC()},
            optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        )

        # Create train and validation datasets
        ds_train = create_dataset(data, labels, train_indices, batch_size)
        ds_val = create_dataset(data, labels, val_indices, batch_size)

        # Define the early stopping callback
        # early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

        # Fit the model with early stopping
        history = model.fit(
            ds_train,
            validation_data=ds_val,
            epochs=100,
            # callbacks=[early_stopping],
        )

        # Plot training metrics for the current fold
        plot_training_metrics(history, model_name, fold)



def create_dataset(data, labels, indices, batch_size):
    # Create dataset from given indices
    ds_data = tf.data.Dataset.from_tensor_slices(tf.gather(data, indices, axis=0))
    ds_labels = tf.data.Dataset.from_tensor_slices(tf.gather(labels, indices, axis=0))
    ds = tf.data.Dataset.zip((ds_data, ds_labels))

    # ds = ds.shuffle(10000)
    ds = ds.batch(batch_size, drop_remainder=False)

    for i, batch in enumerate(ds):
        _, l = batch
        events = sum(l[:,1])
        if events < 1.:
            print(f"Warning, 0 events in batch {i}.")
    
    return ds




def plot_training_metrics(history, model_name, fold):
    epochs = list(range(1, len(history.history['loss']) + 1))

    # Create subplots
    fig = sp.make_subplots(rows=1, cols=2, subplot_titles=("Loss", "Harrell's C"))

    # Create a trace for training loss
    trace_train_loss = go.Scatter(x=epochs,
                                  y=history.history['loss'],
                                  mode='lines',
                                  name='Train Loss')

    # Create a trace for validation loss
    trace_val_loss = go.Scatter(x=epochs,
                                y=history.history['val_loss'],
                                mode='lines',
                                name='Validation Loss')

    # Add loss traces to the subplots
    fig.add_trace(trace_train_loss, row=1, col=1)
    fig.add_trace(trace_val_loss, row=1, col=1)

    # Set the metric key based on the model name
    metric_key = 'risk_harrellsc' if model_name == 'model_att' else 'harrellsc'

    # Create a trace for training Harrell's C
    trace_train_harrells_c = go.Scatter(x=epochs,
                                        y=history.history[metric_key],
                                        mode='lines',
                                        name="Train Harrell's C")

    # Create a trace for validation Harrell's C
    trace_val_harrells_c = go.Scatter(x=epochs,
                                      y=history.history[f'val_{metric_key}'],
                                      mode='lines',
                                      name="Validation Harrell's C")

    # Add Harrell's C traces to the subplots
    fig.add_trace(trace_train_harrells_c, row=1, col=2)
    fig.add_trace(trace_val_harrells_c, row=1, col=2)

    # Update xaxis and yaxis titles
    fig.update_xaxes(title_text='Epoch', row=1, col=1)
    fig.update_yaxes(title_text='Loss', row=1, col=1)
    fig.update_xaxes(title_text='Epoch', row=1, col=2)
    fig.update_yaxes(title_text="Harrell's C", row=1, col=2)

    # Update the layout for the plot
    fig.update_layout(title=f'{model_name} Fold {fold} Training Metrics')

    # Show the plot
    fig.show()



### Training ###

In [10]:
# Split the data into train (80%), val (10%) and test (10%) sets
data_size = data.shape[0]
train_size = int(data_size * 0.9)
indices = np.random.permutation(data_size)
train_indices, test_indices = indices[:train_size], indices[train_size:]

train_data = tf.gather(data, train_indices, axis=0)
test_data = tf.gather(data, test_indices, axis=0)
train_labels = tf.gather(labels, train_indices, axis=0)
test_labels = tf.gather(labels, test_indices, axis=0)

# Use KFold cross-validation on the train set
perform_k_fold_cross_validation(train_data, train_labels, batch_size=64, n_splits=5, model_name='model_att')  # For model_att
# perform_k_fold_cross_validation(train_data, train_labels, model_avg, n_splits=5, model_name='model_avg')  # For model_avg

Fold 1/5
The input shape of model is: (None, None, 49)
The output shape of model is: [(1, 1), (None, None, 1)]
Epoch 1/100


/opt/anaconda3/envs/mil/lib/python3.8/site-packages/tensorflow/python/framework/indexed_slices.py:448: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/cox/cond/gradients/cox/cond/RaggedTile/GatherV2_1_grad/Reshape_1:0", shape=(None,), dtype=int64), values=Tensor("gradient_tape/cox/cond/gradients/cox/cond/RaggedTile/GatherV2_1_grad/Reshape:0", shape=(None,), dtype=float32), dense_shape=Tensor("gradient_tape/cox/cond/gradients/cox/cond/RaggedTile/GatherV2_1_grad/Cast:0", shape=(1,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
/opt/anaconda3/envs/mil/lib/python3.8/site-packages/tensorflow/python/framework/indexed_slices.py:448: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model/input.to_tensor/RaggedToTensor/boolean_mask_1/GatherV2:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model/input.to_tensor/RaggedToTensor/boolean_ma

19/19 [==============================] - 9s 123ms/step - loss: 31.7304 - risk_loss: 31.7304 - risk_harrellsc: 0.4946 - val_loss: 24.8417 - val_risk_loss: 24.8417 - val_risk_harrellsc: 0.4620
Epoch 2/100
19/19 [==============================] - 0s 15ms/step - loss: 29.3565 - risk_loss: 29.3565 - risk_harrellsc: 0.4835 - val_loss: 23.1247 - val_risk_loss: 23.1247 - val_risk_harrellsc: 0.4810
Epoch 3/100
19/19 [==============================] - 0s 14ms/step - loss: 27.8382 - risk_loss: 27.8382 - risk_harrellsc: 0.4851 - val_loss: 22.2803 - val_risk_loss: 22.2803 - val_risk_harrellsc: 0.4922
Epoch 4/100
19/19 [==============================] - 0s 15ms/step - loss: 26.5550 - risk_loss: 26.5550 - risk_harrellsc: 0.4873 - val_loss: 22.0472 - val_risk_loss: 22.0472 - val_risk_harrellsc: 0.4922
Epoch 5/100
19/19 [==============================] - 0s 14ms/step - loss: 25.3943 - risk_loss: 25.3943 - risk_harrellsc: 0.4900 - val_loss: 22.1082 - val_risk_loss: 22.1082 - val_risk_harrellsc: 0.4914
E

Fold 2/5
The input shape of model is: (None, None, 49)
The output shape of model is: [(1, 1), (None, None, 1)]
Epoch 1/100


/opt/anaconda3/envs/mil/lib/python3.8/site-packages/tensorflow/python/framework/indexed_slices.py:448: UserWarning:

Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/cox/cond/gradients/cox/cond/RaggedTile/GatherV2_1_grad/Reshape_1:0", shape=(None,), dtype=int64), values=Tensor("gradient_tape/cox/cond/gradients/cox/cond/RaggedTile/GatherV2_1_grad/Reshape:0", shape=(None,), dtype=float32), dense_shape=Tensor("gradient_tape/cox/cond/gradients/cox/cond/RaggedTile/GatherV2_1_grad/Cast:0", shape=(1,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.

/opt/anaconda3/envs/mil/lib/python3.8/site-packages/tensorflow/python/framework/indexed_slices.py:448: UserWarning:

Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model_1/input.to_tensor_1/RaggedToTensor/boolean_mask_1/GatherV2:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model_1/input.to_tensor_1/RaggedToTensor/boolean_mask/Gat

...

/opt/anaconda3/envs/mil/lib/python3.8/site-packages/tensorflow/python/framework/indexed_slices.py:448: UserWarning:

Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model_3/input.to_tensor_3/RaggedToTensor/boolean_mask_1/GatherV2:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model_3/input.to_tensor_3/RaggedToTensor/boolean_mask/GatherV2:0", shape=(None, 49), dtype=float32), dense_shape=Tensor("gradient_tape/model_3/input.to_tensor_3/RaggedToTensor/Shape:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.

/opt/anaconda3/envs/mil/lib/python3.8/site-packages/tensorflow/python/framework/indexed_slices.py:448: UserWarning:

Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model_3/tf.math.divide_no_nan_3/RaggedTile/Reshape_3:0", shape=(None,), dtype=int64), values=Tensor("gradient_tape/model_3/tf.math.divide_no_nan_3/RaggedTile/Reshape_2:0", shape=(None, 1), dtype

19/19 [==============================] - 5s 100ms/step - loss: 31.5102 - risk_loss: 31.5102 - risk_harrellsc: 0.5305 - val_loss: 22.1587 - val_risk_loss: 22.1587 - val_risk_harrellsc: 0.5354
Epoch 2/100
19/19 [==============================] - 0s 13ms/step - loss: 29.3872 - risk_loss: 29.3872 - risk_harrellsc: 0.5588 - val_loss: 21.8552 - val_risk_loss: 21.8552 - val_risk_harrellsc: 0.3876
Epoch 3/100
19/19 [==============================] - 0s 13ms/step - loss: 28.0353 - risk_loss: 28.0353 - risk_harrellsc: 0.5423 - val_loss: 21.6650 - val_risk_loss: 21.6650 - val_risk_harrellsc: 0.4303
Epoch 4/100
19/19 [==============================] - 0s 13ms/step - loss: 26.7038 - risk_loss: 26.7038 - risk_harrellsc: 0.5415 - val_loss: 21.5673 - val_risk_loss: 21.5673 - val_risk_harrellsc: 0.3399
Epoch 5/100
19/19 [==============================] - 0s 13ms/step - loss: 25.3730 - risk_loss: 25.3730 - risk_harrellsc: 0.5310 - val_loss: 21.4643 - val_risk_loss: 21.4643 - val_risk_harrellsc: 0.3876
E

Fold 5/5
The input shape of model is: (None, None, 49)
The output shape of model is: [(1, 1), (None, None, 1)]
Epoch 1/100


/opt/anaconda3/envs/mil/lib/python3.8/site-packages/tensorflow/python/framework/indexed_slices.py:448: UserWarning:

Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model_4/input.to_tensor_4/RaggedToTensor/boolean_mask_1/GatherV2:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model_4/input.to_tensor_4/RaggedToTensor/boolean_mask/GatherV2:0", shape=(None, 49), dtype=float32), dense_shape=Tensor("gradient_tape/model_4/input.to_tensor_4/RaggedToTensor/Shape:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.

/opt/anaconda3/envs/mil/lib/python3.8/site-packages/tensorflow/python/framework/indexed_slices.py:448: UserWarning:

Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model_4/tf.math.divide_no_nan_4/RaggedTile/Reshape_3:0", shape=(None,), dtype=int64), values=Tensor("gradient_tape/model_4/tf.math.divide_no_nan_4/RaggedTile/Reshape_2:0", shape=(None, 1), dtype

19/19 [==============================] - 7s 101ms/step - loss: 25.9977 - risk_loss: 25.9977 - risk_harrellsc: 0.5280 - val_loss: 32.0133 - val_risk_loss: 32.0133 - val_risk_harrellsc: 0.4619
Epoch 2/100
19/19 [==============================] - 0s 13ms/step - loss: 25.0797 - risk_loss: 25.0797 - risk_harrellsc: 0.5303 - val_loss: 31.8003 - val_risk_loss: 31.8003 - val_risk_harrellsc: 0.4608
Epoch 3/100
19/19 [==============================] - 0s 16ms/step - loss: 24.4665 - risk_loss: 24.4665 - risk_harrellsc: 0.5324 - val_loss: 31.6195 - val_risk_loss: 31.6195 - val_risk_harrellsc: 0.4647
Epoch 4/100
19/19 [==============================] - 0s 13ms/step - loss: 24.0562 - risk_loss: 24.0562 - risk_harrellsc: 0.5336 - val_loss: 31.8857 - val_risk_loss: 31.8857 - val_risk_harrellsc: 0.4486
Epoch 5/100
19/19 [==============================] - 0s 13ms/step - loss: 23.7305 - risk_loss: 23.7305 - risk_harrellsc: 0.5332 - val_loss: 31.7909 - val_risk_loss: 31.7909 - val_risk_harrellsc: 0.4474
E

### Evaluation ###

In [11]:
def evaluate_and_plot_kaplan_meier(model, test_data, test_labels, model_name):
    if model_name == 'model_att':
        risks, _ = model.predict(test_data)
    elif model_name == 'model_avg':
        risks = model.predict(test_data)
    else:
        raise ValueError("Invalid model_name. Choose from 'model_att' and 'model_avg'.")

    cindex = HarrellsC()
    print(f"{model_name} Testing c-index: {cindex(test_labels, risks):.3f}")

    risk_groups = np.squeeze(np.array(risks > np.median(risks), int)) + 1
    km_plot(
        np.array(test_labels),
        groups=risk_groups,
        xlabel="Time",
        ylabel="Survival probability",
        legend=["predicted low risk", "predicted high risk"],
    )

# Evaluate and plot Kaplan-Meier curve for model_att
evaluate_and_plot_kaplan_meier(model=, test_data, test_labels, model_name='model_att')

# Evaluate and plot Kaplan-Meier curve for model_avg
# evaluate_and_plot_kaplan_meier(model_avg, test_data, test_labels, model_name='model_avg')


NameError: name 'model' is not defined